# Basic control structures and variables II

## Why do you only have a hammer anyway?

Yesterday, we covered some central components of Python, such as:

* types, e.g. `int`, `str`

* loops, e.g. `for`, `while`

* modules, e.g. `math`, `numpy`

Today will be a shorter equipment session, followed by more hands on coding. As you should have basic familiarity with coding concepts, I am going to assume object-oriented programming is familiar to you - if not, there is a brief Python-based intro to the key ideas at the bottom ("PS Introduction to objects...")

# Classes

The basic syntax for classes is pretty succinct:

In [ ]:
class MyClass(object):
    some_attribute = "default_value"

    def __init__(self, some_attribute, arg2, arg3):
        # Some initialization (over-riding default)
        self.some_attribute = some_attribute
    
    def some_method(self):
        return self.some_attribute
        
    def another_method(self, arg1):
        if arg1 == "example":
            output = self.some_method()
            return output

A few points to note: `class` is just a block like any other. Default values can be supplied at its top level, but be careful - often it is better to set defaults in the initializer. Methods are defined as functions, but inside the class block. All of them have a first argument `self`, which is the object itself. Like, say, `this` in C++ (but without dereferencing required).

Python classes don't have a particularly strong approach to private/public access, but starting with an underscore is a conventional approach to indicate a member shouldn't be touched by anything outside the class. We subclass from `object` - additional classes may be added by comma-separation. Subclassing from `object` is actually the default in Python3 if no parentheses (or parent class) are provided - so-called *new-style classes*.

You also see your first magic method - the `__init__` function. This is the constructor, called when the class is created. Here we use it to override a default from the arguments it is passed.

In [ ]:
aninstance = MyClass("non-default value", 1, 2)

Here, you see the arguments to the `__init__` function.

In [ ]:
aninstance.another_method("example")

And, one method has called the next and returned the value we asked. We can check the type and dir of this like any other object.

In [ ]:
print(type(aninstance))
print(", ".join(dir(aninstance)))

Note that you can call a override a method in a parent class (for example `a_method` in `MyParentClass`) by defining a method with the same name. If you want to effectively wrap it, you can use the format

```python
class MyClass(MyParentClass):
    ...
    def a_method(self, arg1, arg2):
        val = MyParentClass.mymethod(self, arg1, arg2)
        # Do something to val
        return val
```

Note that you must explicitly pass "`self`" in this case.

## Exercise

In [ ]:
class Institution:
    location = "(unknown)"
    
    def __init__(self, location):
        self.location = location
        
    def __str__(self):
        return "Institution in %s" % self.location
        

In [ ]:
my_institution = Institution("Paris")
str(my_institution)

In [ ]:
class University(Institution):
    ...

In [ ]:
my_institution = University("Belfast")
your_institution = University("Paris")
str(my_institution) == "University in Belfast" and str(your_institution) == "University in Paris"

Can you override the `__str__` magic method so that the final cell prints True?

# Exceptions
## Are something extraordinary

In the words of one of early computing's most idiosyncratic legends, Rear Admiral Dr Grace Murray Hopper, "It is often easier to ask for forgiveness than to ask for permission". A distinguishing feature of Python is that it is built from the ground up with this maxim in mind, known as EAFP. In practice, this means preferring exceptions over tests, so using a `try-except` block (or, elsewhere, `try-catch`, etc.) instead of an `if` statement when checking whether you can perform an action.

Simple examples where you are more likely to use an exception in Python than another language:

* When making a directory

Can you see why this might be useful? If you use an `if` to check for non-existence, there is a potential race condition: by the time you reach the body, it could be made. If you catch the exception, you know it existed exactly when you tried to make it. (Although, the recommended routine, `os.makedirs`, has an optional don't-complain-if-dir-exists argument, which is probably even better)

* When testing file existence

Again with the race condition - this is in fact a security issue, as an attacker can create the file between you checking for existence and opening it for writing. If they create it, of course, then they set the permissions and can see the content regardless of your attempts to block reading.

* Checking type

We talked a lot about duck-typing... the underlying principle is that you never reject input types *as long as they work*. Now imagine you have a routine with an argument `x`, where you want to do one thing if `x` is numeric type and another if it's, say, a string... if you use `if` and check their type, well what if this is some weird subclass of float that your routine has been sent, or the author of this type has carefully implemented all the necessary magic functions to make it quack like a float, but it's a completely unrelated class?

Push the boat out and see if it floats. Try casting to a float and if it doesn't work catch the exception. Then try casting to a string. Everything should cast to a string somehow. Now you have made checking numeric-ness a problem of `float`, which is infinitely more qualified to do this than you are.

* Checking for a dictionary key

Maybe what you think is a dictionary isn't - it's something that, when you request `thingy["something"]` will check whether `"something"` is something it might dynamically add, and, if so, will gladly return the result. If you check first (`if "something" in thingy:`), either the answer is misleading ("no"), or what looks like an innocent if-clause is modifying your dictionary. Moreover, even for an ordinary dictionary, having an if-clause followed by a retrieval hits your dictionary scan twice - try-catch only searches once.

Hopefully this motivates the idea of exceptions before tests - EAFP. Why then have you been repeatedly told not to do this in, as Python calls them, LBYL (Look Before You Leap) languages? The answer is usually that exceptions are horrendously slow and inefficient. In Python this isn't true, by design. They are so fundamental to the language that every loop in fact ends, not with a failing test, but when the iterable throws a specific exception (the `StopIteration` exception).

# Try a try

Here is an example of a piece of code that throws an exception:

In [ ]:
something_stupid()
print("Next step")

Python provides lots of useful information about an _exception_ by default - in this case, that `something_stupid` is not defined. However, the default approach interrupts the flow - "Next step" is never printed. Sometimes, we need a more nuanced approach, where we can make decisions about an exception within our code.

The actual syntax is similar to what you will likely have seen in other languages:

In [ ]:
try:
    something_stupid()
except:
    print('Doh!')
print('Next step')

Well, the first thing is that we didn't define *something_stupid*. So we get an exception and our extremely generic `except` provides no useful information.

However, "Next step" has been printed - our try-except block has caught the exception, and allowed the flow to continue onto the next command.

## An improvement

Realistically, when catching an exception, you probably want to do something about it. Perhaps print an explanatory error message.

In [ ]:
try:
    something_stupid()
except Exception as e:
    print('Doh! You forgot that', e)

Definitely better, and note that `print` will cast `e`, the exception, to a string - it is actually a more complex object:

In [ ]:
try:
    something_stupid()
except Exception as e:
    print(', '.join(dir(e)))

Note in particular that `e` can have arguments when it is thrown - retrieved via `e.args`. Suppose we don't actually want to stop the exception bubbling up, just to some logging or tidy-up on the way through.

In [ ]:
try:
    something_stupid()
except Exception as e:
    print('I am *not* cleaning your mess for you, deal with it yourself!')
    raise e

Now we get the same exception we would have got the last time, but the `raise` keyword has kept it moving on past. This is quite useful, as Python exceptions tend to have lots of juicy info we wouldn't want to lose by ending our except block with a boring `print('oh noes')` and program exit.

In fact this is how we can throw a brand new exception - we take one of the standard types, e.g. "`RuntimeError`" and `raise` it with an explanatory argument:

In [ ]:
raise RuntimeError("Something quite generic went wrong")

# Better yet

This is a very broad error, and a parent class of most of the exceptions we have seen. It doesn't make it easy for calling functions to pin-point what went wrong...

Looking at it from the other end, just as we should be more specific about what we catch, we should be more specific about what we catch. This is a good idea, as, usually when we catch an exception, it is because we expect a particular thing to go wrong. As a case in point:

In [ ]:
try:
    os.makedirs(dirname)
except:
    # Great, someone has already created that directory
    # We can carry on!
    pass
# lalalalala...
print("just mucking about with my friend dirname")

First note `pass`. This is required because every block must have at least one non-comment line - if nothing else is there, we can use `pass` - it is Python's `no-op` if that helps.

Now ask yourself, "did we ever actually define dirname?" No? Then as soon as we use it after our supposed check, we will get an unhandled NameError exception.

We can specify what type of exceptions we catch... dir-already-exists have type `OSError`.

In [ ]:
try:
    os.makedirs(dirname)
except OSError:
    print("We had an error creating the directory")
# lalalalala...
print("just mucking about with my friend dirname")

Now we have stepped out of the way of genuine errors coming through. We can actually handle different exception types in different ways - there are plenty of cases this might be useful, if you want to do something awkward which could fail in five different directions.

In [ ]:
try:
    os.makedirs(dirname)
except OSError:
    # Great, someone has already created that directory
    # We can carry on!
    pass
except NameError:
    print("""
    This is the third code example
    where you haven't defined `dirname`.
    Seriously, catch yourself on.
    """)
# lalalalala...
print("just mucking about with my friend dirname")

Yeah. Great.

Useful tool just slotted in there: multi-line strings. If you start a string with three quotes, you can keep going on and on until you hit another three.

One final example to show how, for this particular case, you really can be a little more specific.

In [ ]:
dirname = "/etc/passwd"
try:
    os.makedirs(dirname)
except OSError:
    # Great, someone has already created that directory
    # We can carry on!
    pass
# lalalalala...
print("just mucking about with my friend", dirname)

Emm...we tried to make a directory with the same name as a key system file and are blithely assuming that every failure in doing so is simply because it is an existing directory. Not so good.

In [ ]:
import errno

dirname = "/etc/pass"

try:
    os.makedirs(dirname)
except OSError as e:
    if e.errno != errno.EEXIST:
        raise
    # Great, someone has already created that directory
    # We can carry on!
# lalalalala...
print("just mucking about with my friend", dirname)

This emphasises the fact that subclasses of `Exception` often have additional contextually-relevant properties, which you should use. It also points out that `if` statements still have their place in error-handling!

When `raise` has no argument, it re-raises whichever exception it was that got us into this mess.

# A few important Exceptions
## That you might want to catch

* KeyError - `box_of_tricks["not here"]`
* TypeError - `1 + "banana"`
* IOError - `open('/etc/passwd', 'w')`


Exceptions are often more *pythonic* than pre-checking. [This Python2 doc](https://docs.python.org/2/howto/doanddont.html#exceptions) is a good start on that road. CRUCIAL READING!


# Exercise

...if we have time...

Below, we subclass `str` and call it `MyString`. It has one method, called `join` to override the usual `join` functionality that strings have - otherwise, `MyString` strings behaves exactly as any other Python strings.

Exercise: make this overridden `join` method check if the iterable you are joining has "Pottery" in it and, if so, make it raise a RuntimeError.

Extension: combine this with your previous dictionary-to-strings function. Then update your dictionary-to-strings function to have a try-except block, which quietly swaps "Pottery" for "More Python" when the RuntimeError above error occurs (and then continues as normal).

In [ ]:
class MyString(str):
    def join(self, iterable):
        # Replace this with your Pottery-checker statement
        # ...
        # then continue as normal:
        return str.join(self, iterable)
    
delimiter = MyString(",")
delimiter.join(my_meetup_dot_com_profile["Interests"])

# Tuples
## The Ice-List

Tuples are basically frozen lists...

In [ ]:
target_coordinates = (56, -5)
print(target_coordinates[0], "N", target_coordinates[1], "E")

...but it's fixed, so you can't do this...

In [ ]:
target_coordinates[1] = 38

If you alter it to set the whole variable (try it), that's fine though - you can alter what `target_coordinates` points to, you just can't alter the tuple itself.

Lists are constantly changing - length, content, type of content. Really, they aren't much like arrays in compiled languages like C or FORTRAN. They are far too flightly to be used as something like a dictionary index for example. However, supose I have a pair - exactly two elements, one a known `int` and one a known `string`, and it can't change. Well, if you stick two basic types together then why can't you do the same thing with it you can do with basic types? There's no weird changing behaviour going on, so if you freeze a list into something like that, can you use it as a dictionary index?

In short - yes you can!

In [ ]:
battleships = {}  # new dict
coordinates = [3, 2]
battleships[coordinates] = "HIT"

So it doesn't work with a list - try it now with a tuple - swap the square brackets on the *second* line for parens () and re-run

In [ ]:
print("At", coordinates, "we have a", battleships[coordinates])

**Aside**: A *hashable* type can be used as an index in a dict - tuples are, lists aren't (because they can change, aka are *mutable*)

# Lambdas & Generators
## Functional Approaches

These are best understood in practice but I will give a couple of definitions.

__lamdba__ : a very short bit of code, a single action, used as a function

In [ ]:
mylambda = (lambda x: x + 18)
mylambda(1)

We don't really need the parens on the first line, but they are just to show you that the RHS is entirely a single entity - a lambda. It is assigned to the variable `mylambda`.

Lambdas are handy for passing around very short functions that return a useful value within one single statement. For example, passing a sorting function to a sorting algorithm, or a filtering function that returns a boolean to a data pruning method. In fact, any function, defined with `def` can be assigned to a variable and passed around.

In [ ]:
sentence = ["this sentence", "is", "really well sorted"]
key_function = lambda s: s.count(' ')
sorted(sentence, key=key_function)

In [ ]:
dictionary = {'me': 'myself', 'you': 'yourself'}
list(map(lambda pair: pair[0] + pair[1], dictionary.items()))

### Exercise

Can you write a lambda to filter out all numbers divisible by 3 _or_ more than 3 digits long?

In [ ]:
filter_function = lambda n: ???? # INSERT FUNCTION HERE - SHOULD EVALUATE TO TRUE or FALSE
list(filter(filter_function, [1, 234, 129123, 15, 34, 41991, 14])) # Should give [1, 34, 14]

### Generators

These are __loops inside lists__ ...

This expands on the idea of iterables from yesterday - entities that churn out successive values, whether strings giving characters, lists of objects, ranges of numbers. So far, all of those have been pre-known before we loop over them. But why? Couldn't we have something that chucks out values as we keep asking?

In [ ]:
mylist = [x * 2 for x in range(10)]
mylist

In [ ]:
from math import pi, sin
[sin(pi * x) for x in [0, 0.5, -0.5]]

These can be especially useful when combined with lambdas, or short multiline functions.

# Wrap-up

This has covered (briefly):

* classes

* exceptions

* lambdas

* generators

We haven't talked about a number of structures, but a couple you may come across that we don't cover in this course, are asynchronous routines (`async`, `await`) that follow the coroutine paradigm; and decorators, which allow us to mark certain functions and methods to be wrapped by a function-of-a-function. This looks like:

```python
@mydecorator
def afunction():
  ...
```  

In this case `mydecorator` will somehow modify the function's attributes or effects - examples are `@coroutine` for marking a function as async, `@staticmethod` for marking a class's method as static, and `@timer` for timing a function when it is called.

# PS: an introduction to objects...

# Objects

## Totally class

If you haven't done object oriented coding before, I am afraid I am not going to give you the magic bullet now. It is an important concept to get to grips with, but for the moment, we will keep it fairly functional. Do take time to look through it later - Python is an excellent language for playing around with it and getting an intuition for what an object is. A number of you will be very familiar with this so please bear with me and the very rough introductory description I will give - for those who are not so familiar, this is worth getting the basics down early.

In real life, everything has properties, and many things have things they can do. In Python these properties are called attributes and the things an object can do are called methods. Together these are called members and, in Python, are accessed by putting a dot after the object and the attribute or method name. Everything else is just like any other variable or function, respectively.

For instance, my dog Freddie has properties - he has a colour, which is black (technically, he is imaginary, but in my head he is definitely black). In Python terms, this would be:

**Attribute**: `freddie.colour = black`

I can tell Freddie to roll-over. When I do so, I am, in some sense, calling Freddie's method:

**Method**: `freddie.roll_over()`

I have another dog, Nitwit. Nitwit can also roll over...

**Method**: `nitwit.roll_over()`

Both dogs can shake hands, but I need to tell them which paw...

**Method**: `freddie.shake_paw('left')`

At this point, you're probably wondering what exactly *is* the set of attributes and methods that my dogs have? This template, showing what attributes and methods a dog of mine can be expected to have, is called a class.

# A simple class

In [ ]:
class PhilsDog:
    name = ""
    colour = ""
    def shake_paw(self, side):
        print("My name is", self.name, "and I am shaking my", side, "paw like a good dog")

Here we create a class called `PhilsDog` that all of my dogs *implement* (that is, they are of that type). It indicates that they will have a colour and that they have a method called `shake_paw`. Think of this from the perspective of the dog - the first argument, `self`, is a little Python magic that refers to the dog itself. This lets the dog use it's name and colour (and any other methods) in the `shake_paw` method. The second argument is which paw I told my dog to shake. In response, any dog of mine says "Shaking left/right paw like a good dog". That's quite impressive, but I'd rather they just learned to zip it and actually shake their paws instead.

Now, that's just the template for one of my dogs, so how do I use it?

In [ ]:
freddie = PhilsDog()
freddie.name = "Freddie"
freddie.colour = "black"

nitwit = PhilsDog()
nitwit.name = "Nitwit"
nitwit.colour = "brown"

print("Freddie is", freddie.colour, "while Nitwit is", nitwit.colour)

I call the class like a function. This creates a new PhilsDog object - in computing terminology, I **instantiate** the class, creating a new **instance** of PhilsDog. Freddie is an instance of PhilsDog and so is Nitwit. But PhilsDog is just a template - as you can see, both Freddie and Nitwit have their own colour and name. I can update these just as any variable, and Freddie's doesn't affect Nitwit's, and I can read both back out again.

In [ ]:
freddie.shake_paw('left')
nitwit.shake_paw('right')

Good dogs. Here we see the `shake_paw` method being called for each dog and with a different `side` parameter. To remind you, the body of the method was:

```python
    def shake_paw(self, side):
        print("My name is", self.name, "and I am shaking my", side, "paw like a good dog")
```

You can see how the `self.name` matches the name I gave each dog on the previous slide.

If that was familiar to you, then that was probably a very unexciting few minutes - if not, it's probably come and gone very quickly. Thankfully, we only need to know that, if we have an object, we can get its attributes and methods by adding a dot and the attribute/method name.

Now why is that important...

# In Python
## Everything is an object (nearly)

In [ ]:
"just a normal string".upper()

In [ ]:
"Another string".islower()

# Two useful tools

In [ ]:
type(freddie)

"`type`" lets you examine what class an object is. Don't worry about that `__main__` just for the moment.

In [ ]:
dir(freddie)[-3:]

This is a list of all the members of `freddie`. If he hasn't had any added on the fly (which is possible in Python), this is the same as the members of the `PhilsDog` class. That syntax in the brackets is coming up in a couple of slides, but basically it means, the last three items. Python denotes somewhat magic functions with double-underscores on either side (they are all the previous items I'm hiding away) - there is good reason for these being here, but they aren't essential just now. Try removing the bit in brackets, including the brackets, to see what you get.